In [1]:
import gc
import time
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.preprocessing
from datagenerators import DataGeneratorBars
from sklearn.metrics import (
    balanced_accuracy_score,
    confusion_matrix,
    plot_confusion_matrix,
)

In [2]:
import tensorflow
from tensorflow import keras

print("tensorflow", tensorflow.__version__)
print("keras", keras.__version__)

tensorflow 2.3.1
keras 2.4.0


In [3]:
max_neutrons = 4
config = {
    "distance": 15,
    "doubleplane": 30,
    "energy": 600,
    "erel": 500,
    "neutrons": range(1, max_neutrons + 1),
    "subruns": range(6),  # range(19),
    "subrun_cache_size": 6,
    "batch_size": 100,
    #"label": "nPN",
    "label": "nPH",
    "mode": "split",
    "suffix": "bars-scaled.parquet"
    #"suffix": "bars.parquet"
}

validation_config = config.copy()
validation_config["subruns"] = [19]

In [4]:
generator = DataGeneratorBars(config)
validation_generator = DataGeneratorBars(validation_config)

Rows in one file: 10000
400 batches per subrun
2400 total batches in [range(0, 6)] caches
Loading subruns range(0, 6) for cache 0
Rows in one file: 10000
400 batches per subrun
400 total batches in [[19]] caches
Loading subruns [19] for cache 0


In [5]:
from tensorflow.keras.layers import Dense, Input, concatenate, Dropout
from tensorflow.keras.models import Model

# define two sets of inputs
inputTri = Input(shape=(3,))
inputBars = Input(shape=(30 * 100 * 2,))

# the first branch operates on the first input
t = Dense(20, activation="relu")(inputTri)
t = Model(inputs=inputTri, outputs=t)

# the second branch opreates on the second input
b = Dense(20, activation="relu")(inputBars)
b = Model(inputs=inputBars, outputs=b)

# combine the output of the two branches
combined = concatenate([t.output, b.output])
z = Dense(10, activation="relu")(combined)
z = Dense(max_neutrons + 1, activation="softmax")(combined)

model = Model(inputs=[t.input, b.input], outputs=z)

optimizer = keras.optimizers.Adam
loss = keras.losses.CategoricalCrossentropy
model.compile(loss=loss(), optimizer=optimizer(), metrics=["accuracy"])

In [6]:
history = model.fit(generator, verbose=2, epochs=20, shuffle=False, validation_data=validation_generator, batch_size=1000)

Epoch 1/20


InternalError:  Blas GEMM launch failed : a.shape=(100, 6000), b.shape=(6000, 20), m=100, n=20, k=6000
	 [[node functional_5/dense_1/MatMul (defined at <ipython-input-6-84f42f43ddeb>:1) ]] [Op:__inference_train_function_807]

Function call stack:
train_function


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Test"], loc="upper left")
plt.show()

# Plot training & validation loss values
plt.plot(history.history["loss"])
# plt.plot(history.history["val_loss"])
plt.title("Model loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train", "Test"], loc="upper left")
plt.show()

In [ ]:
#del generator
#del validation_generator

In [ ]:
# gc.collect()
# test_config = config.copy()
# test_config["subruns"] = range(14, 17)
# test_generator = DataGeneratorBars(test_config)

In [ ]:
#X = test_generator.features
X = validation_generator.features
#y_true = np.argmax(test_generator.labels, axis=1)
y_true = np.argmax(validation_generator.labels, axis=1)
y_pred = np.argmax(model.predict(X), axis=1)

In [ ]:
from sklearn.metrics import (
    balanced_accuracy_score,
    confusion_matrix,
    plot_confusion_matrix,
)

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

bac = balanced_accuracy_score(y_true, y_pred)
print(bac)
cm = confusion_matrix(y_true, y_pred, labels=range(0, len(config["neutrons"]) + 1))
print(cm)
cmrel = confusion_matrix(y_true, y_pred, labels=range(0, len(config["neutrons"]) + 1), normalize="true")
print((cmrel * 100).round())